In [1]:
import json

alpha_vantage_api_key = None

with open("../config.json") as config:
    alpha_vantage_api_key = json.load(config)["alpha_vantage_api_key"] 

alpha_vantage_api_key

'MYSVBQMG0HY6KDVI'

In [23]:
# https://www.alphavantage.co/documentation/
import requests

'''
    Returns price-history for a single
    ticker as JSON
'''
def _get_data(ticker_symbol):
    default_request_params = {
        #"symbol": None, # required
        "function": "TIME_SERIES_DAILY",
        "outputsize": "compact",
        "datatype": "json",
        "apikey": alpha_vantage_api_key
    }
    
    request_params = dict( default_request_params )
    request_params["symbol"] = ticker_symbol
    
    response = requests.get("https://www.alphavantage.co/query?", params=request_params)
    
    return response.json()

In [34]:
'''
    Returns price-history for given
    tickers as a datetime-indexed dataframe
'''
def get_data(list_of_ticker_symbols):
    master_df = pd.DataFrame()

    for ticker_symbol in list_of_ticker_symbols:
        data = _get_data(ticker_symbol)
        data_as_df = pd.DataFrame.from_dict( data["Time Series (Daily)"], orient='index' ) # !! flagged for unoptimized 

        # extract closing price
        master_df[ticker_symbol] = data_as_df["4. close"].apply( lambda price : float(price) )  
    
    return master_df

In [35]:
# Example 
example_stock_list = "CHK AMD GE UBER BAC GDX BABA KR FIT GOLD AAPL M MS SIRI MSFT".split(" ")



example_df = get_data(example_stock_list[:3])

example_df

,CHK,AMD,GE
2019-06-17,1.830,29.200,10.050
2019-06-18,1.900,30.450,10.420
2019-06-19,1.830,30.500,10.340
2019-06-20,1.980,30.010,10.630
2019-06-21,1.950,29.100,10.480
2019-06-24,1.910,29.260,10.280
2019-06-25,1.830,28.860,10.220
2019-06-26,1.910,29.920,10.270
2019-06-27,1.880,30.740,10.400
2019-06-28,1.950,30.370,10.500


In [39]:
example_df.corr()

,CHK,AMD,GE
CHK,1.000000,0.138386,0.640583
AMD,0.138386,1.000000,0.453216
GE,0.640583,0.453216,1.000000
